<a href="https://colab.research.google.com/github/Rohanrathod7/my-ml-labs/blob/main/17_Introduction_to_Natural_Language_Processing_in_Python/04_Building_a_%22fake_news%22_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 4. Informed Search


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime as dt
# Import confusion matrix and train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LogisticRegression, LinearRegression
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier

url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/15_Hyperparameter_Tuning_in_Python/Dataset/results_df.csv"
# Read the CSV file
# Apply pd.to_numeric only to relevant columns, excluding 'text'
results_df = pd.read_csv(url)


display(results_df.head())

,max_depth,min_samples_leaf,learn_rate,accuracy
0,4,16,0.624362,95
1,10,14,0.477450,97
2,7,14,0.050067,96
3,5,12,0.023356,96
4,6,12,0.771275,97


### Classifying fake news using supervised learning with NLP

**Which possible features?**  
Which of the following are possible features for a text classification problem?

    -> Number of words in a document.

    Specific named entities.

    Language.

**Training and testing**   
What datasets are needed for supervised learning?

    -> Both training and testing data.

**CountVectorizer for text classification**  
It's time to begin building your text classifier! The data has been loaded into a DataFrame called df. Explore it in the IPython Shell to investigate what columns you can use. The .head() method is particularly informative.

In this exercise, you'll use pandas alongside scikit-learn to create a sparse text vectorizer you can use to train and test a simple supervised model. To begin, you'll set up a CountVectorizer and investigate some of its features.



Download data -> https://s3.amazonaws.com/assets.datacamp.com/production/course_3629/fake_or_real_news.csv

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# Google Drive shared file link
file_id = '1uVtIvPVw4br2Yan_7Ew09wnnyYIJXOwG'

# Construct direct download URL
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Load CSV file while skipping problematic lines
df = pd.read_csv(download_url, on_bad_lines='skip', encoding='utf-8', engine='python')

# Display first 5 rows
df.head()


,Unnamed: 0,title,text,label,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Import the necessary modules
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Print the head of df
print(df.head())

# Create a series to store the labels: y
y = df.label

# Fill missing values in the 'text' column with an empty string
df['text'] = df['text'].fillna('')

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.33, random_state=53)

# Initialize a CountVectorizer object: count_vectorizer
count_vectorizer = CountVectorizer(stop_words='english')

# Transform the training data using only the 'text' column values: count_train
count_train = count_vectorizer.fit_transform(X_train)

# Transform the test data using only the 'text' column values: count_test
count_test = count_vectorizer.transform(X_test)

# Print the first 10 features of the count_vectorizer
print(count_vectorizer.get_feature_names_out()[:10])

  Unnamed: 0                                              title  \
0       8476                       You Can Smell Hillary’s Fear   
1      10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2       3608        Kerry to go to Paris in gesture of sympathy   
3      10142  Bernie supporters on Twitter erupt in anger ag...   
4        875   The Battle of New York: Why This Primary Matters   

                                                text label Unnamed: 4  \
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE        NaN   
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE        NaN   
2  U.S. Secretary of State John F. Kerry said Mon...  REAL        NaN   
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE        NaN   
4  It's primary day in New York and front-runners...  REAL        NaN   

  Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ... Unnamed: 131  \
0        NaN        NaN        NaN        NaN        NaN  ...          NaN   
1 

**TfidfVectorizer for text classification**  
Similar to the sparse CountVectorizer created in the previous exercise, you'll work on creating tf-idf vectors for your documents. You'll set up a TfidfVectorizer and investigate some of its features.

In this exercise, you'll use pandas and sklearn along with the same X_train, y_train and X_test, y_test DataFrames and Series you created in the last exercise.



In [5]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize a TfidfVectorizer object: tfidf_vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# Transform the training data: tfidf_train
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data: tfidf_test
tfidf_test = tfidf_vectorizer.transform(X_test)

# Check the method used to get feature names
print(tfidf_vectorizer.get_feature_names_out()[:10])

# Print the first 5 vectors of the tfidf training data
print(tfidf_train[:5].toarray())

['00' '000' '0000' '000000031' '00000031' '0001' '0002' '000ft' '000x'
 '001']
[[0.         0.06262927 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


**Inspecting the vectors**  
To get a better idea of how the vectors work, you'll investigate them by converting them into pandas DataFrames.

Here, you'll use the same data structures you created in the previous two exercises (count_train, count_vectorizer, tfidf_train, tfidf_vectorizer) as well as pandas, which is imported as pd.

In [6]:
# Create the CountVectorizer DataFrame: count_df
count_df = pd.DataFrame(count_train.toarray(), columns=count_vectorizer.get_feature_names_out())

tfidf_df = pd.DataFrame(tfidf_train.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Print the head of count_df
print(count_df.head())

# Print the head of tfidf_df
print(tfidf_df.head())

# Calculate the difference in columns: difference
difference = set(count_df.columns) - set(tfidf_df.columns)
print(difference)

# Check whether the DataFrames are equal
print(count_df.equals(tfidf_df))

   00  000  0000  000000031  00000031  0001  0002  000ft  000x  001  ...  שתי  \
0   0    7     0          0         0     0     0      0     0    0  ...    0   
1   0    0     0          0         0     0     0      0     0    0  ...    0   
2   0    0     0          0         0     0     0      0     0    0  ...    0   
3   0    0     0          0         0     0     0      0     0    0  ...    0   
4   0    0     0          0         0     0     0      0     0    0  ...    0   

   תאמצנה  תוצאה  תחל  תיירות  תנותק  תעודת  תתרכז  عربي  ยงade  
0       0      0    0       0      0      0      0     0      0  
1       0      0    0       0      0      0      0     0      0  
2       0      0    0       0      0      0      0     0      0  
3       0      0    0       0      0      0      0     0      0  
4       0      0    0       0      0      0      0     0      0  

[5 rows x 56478 columns]
    00       000  0000  000000031  00000031  0001  0002  000ft  000x  001  \
0  0.0  0.0626

**Text classification models**  
Which of the below is the most reasonable model to use when training a new supervised model using text vector data?

    -> Naive Bayes

**Training and testing the "fake news" model with CountVectorizer**   
Now it's your turn to train the "fake news" model using the features you identified and extracted. In this first exercise you'll train and test a Naive Bayes model using the CountVectorizer data.

The training and test sets have been created, and count_vectorizer, count_train, and count_test have been computed

In [12]:
# Import the necessary modules
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import numpy as np

# Instantiate a Multinomial Naive Bayes classifier: nb_classifier
nb_classifier = MultinomialNB()

# Filter out NaN values from y_train and count_train
nan_mask_train = y_train.isna()
nan_indices_train = np.where(~nan_mask_train)[0]
count_train_filtered = count_train[nan_indices_train]
y_train_filtered = y_train.iloc[nan_indices_train]

# Filter out NaN values from y_test and count_test
nan_mask_test = y_test.isna()
nan_indices_test = np.where(~nan_mask_test)[0]
count_test_filtered = count_test[nan_indices_test]
y_test_filtered = y_test.iloc[nan_indices_test]


# Fit the classifier to the filtered training data
nb_classifier.fit(count_train_filtered, y_train_filtered)

# Create the predicted tags: pred
pred = nb_classifier.predict(count_test_filtered)

# Calculate the accuracy score: score
score = metrics.accuracy_score(y_test_filtered, pred)
print(score)

# Calculate the confusion matrix: cm
cm = metrics.confusion_matrix(y_test_filtered, pred, labels=['FAKE', 'REAL'])
print(cm)

0.8408476104598738
[[906 146]
 [ 79 959]]


**Training and testing the "fake news" model with TfidfVectorizer**  
Now that you have evaluated the model using the CountVectorizer, you'll do the same using the TfidfVectorizer with a Naive Bayes model.

The training and test sets have been created, and tfidf_vectorizer, tfidf_train, and tfidf_test have been computed. Additionally, MultinomialNB and metrics have been imported from, respectively, sklearn.naive_bayes and sklearn.

In [14]:
# Create a Multinomial Naive Bayes classifier: nb_classifier
nb_classifier = MultinomialNB()

# Filter out NaN values from y_train and tfidf_train
nan_mask_train = y_train.isna()
nan_indices_train = np.where(~nan_mask_train)[0]
tfidf_train_filtered = tfidf_train[nan_indices_train]
y_train_filtered = y_train.iloc[nan_indices_train]

# Filter out NaN values from y_test and tfidf_test
nan_mask_test = y_test.isna()
nan_indices_test = np.where(~nan_mask_test)[0]
tfidf_test_filtered = tfidf_test[nan_indices_test]
y_test_filtered = y_test.iloc[nan_indices_test]


# Fit the classifier to the filtered training data
nb_classifier.fit(tfidf_train_filtered, y_train_filtered)

# Create the predicted tags: pred
pred = nb_classifier.predict(tfidf_test_filtered)

# Calculate the accuracy score: score
score = metrics.accuracy_score(y_test_filtered, pred)
print(score)

# Calculate the confusion matrix: cm
cm = metrics.confusion_matrix(y_test_filtered, pred, labels=['FAKE', 'REAL'])
print(cm)

0.7799819657348963
[[ 713  339]
 [  21 1017]]


**Improving the model**  
What are possible next steps you could take to improve the model?


      Tweaking alpha levels.

      Trying a new classification model.

      Training on a larger dataset.

      Improving text preprocessing.

**Improving your model**   
Your job in this exercise is to test a few different alpha levels using the Tfidf vectors to determine if there is a better performing combination.

The training and test sets have been created, and tfidf_vectorizer, tfidf_train, and tfidf_test have been computed.

In [16]:
# Create the list of alphas: alphas
alphas = np.arange(0, 1, 0.1)

# Filter out NaN values from y_train and tfidf_train
nan_mask_train = y_train.isna()
nan_indices_train = np.where(~nan_mask_train)[0]
tfidf_train_filtered = tfidf_train[nan_indices_train]
y_train_filtered = y_train.iloc[nan_indices_train]

# Filter out NaN values from y_test and tfidf_test
nan_mask_test = y_test.isna()
nan_indices_test = np.where(~nan_mask_test)[0]
tfidf_test_filtered = tfidf_test[nan_indices_test]
y_test_filtered = y_test.iloc[nan_indices_test]

# Define train_and_predict()
def train_and_predict(alpha):
    # Instantiate the classifier: nb_classifier
    nb_classifier = MultinomialNB(alpha=alpha)
    # Fit to the training data
    nb_classifier.fit(tfidf_train_filtered, y_train_filtered)
    # Predict the labels: pred
    pred = nb_classifier.predict(tfidf_test_filtered)
    # Compute accuracy: score
    score = metrics.accuracy_score(y_test_filtered, pred)
    return score

# Iterate over the alphas and print the corresponding score
for alpha in alphas:
    print('Alpha: ', alpha)
    print('Score: ', train_and_predict(alpha))
    print()

Alpha:  0.0


/usr/local/lib/python3.11/dist-packages/sklearn/naive_bayes.py:898: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/usr/local/lib/python3.11/dist-packages/sklearn/naive_bayes.py:898: RuntimeWarning: invalid value encountered in subtract
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(


Score:  0.0

Alpha:  0.1
Score:  0.8503155996393147

Alpha:  0.2
Score:  0.8381424706943192

Alpha:  0.30000000000000004
Score:  0.8309287646528404

Alpha:  0.4
Score:  0.8219116321009918

Alpha:  0.5
Score:  0.8133453561767358

Alpha:  0.6000000000000001
Score:  0.8065825067628494

Alpha:  0.7000000000000001
Score:  0.7984670874661858

Alpha:  0.8
Score:  0.7899008115419297

Alpha:  0.9
Score:  0.7871956717763751



**Inspecting your model**  
Now that you have built a "fake news" classifier, you'll investigate what it has learned. You can map the important vector weights back to actual words using some simple inspection techniques.

You have your well performing tfidf Naive Bayes classifier available as nb_classifier, and the vectors as tfidf_vectorizer.

In [26]:
# Get the class labels: class_labels
class_labels = nb_classifier.classes_

# Extract the features: feature_names
feature_names = tfidf_vectorizer.get_feature_names_out()

# Zip the feature names together with the log probabilities for the first class and sort: feat_with_log_probs_0
feat_with_log_probs_0 = sorted(zip(nb_classifier.feature_log_prob_[0], feature_names))

# Zip the feature names together with the log probabilities for the second class and sort: feat_with_log_probs_1
feat_with_log_probs_1 = sorted(zip(nb_classifier.feature_log_prob_[1], feature_names))


# Print the first class label and the top 20 feat_with_log_probs_0 entries (lowest log probability)
print(class_labels[0], feat_with_log_probs_0[:20])
print("\n")
# Print the second class label and the bottom 20 feat_with_log_probs_1 entries (highest log probability)
print(class_labels[1], feat_with_log_probs_1[-20:])

  [(np.float64(-10.941606460775276), '00'), (np.float64(-10.941606460775276), '000'), (np.float64(-10.941606460775276), '0000'), (np.float64(-10.941606460775276), '000000031'), (np.float64(-10.941606460775276), '00000031'), (np.float64(-10.941606460775276), '0001'), (np.float64(-10.941606460775276), '0002'), (np.float64(-10.941606460775276), '000ft'), (np.float64(-10.941606460775276), '000x'), (np.float64(-10.941606460775276), '001'), (np.float64(-10.941606460775276), '003'), (np.float64(-10.941606460775276), '004'), (np.float64(-10.941606460775276), '005'), (np.float64(-10.941606460775276), '00684'), (np.float64(-10.941606460775276), '006s'), (np.float64(-10.941606460775276), '007'), (np.float64(-10.941606460775276), '007s'), (np.float64(-10.941606460775276), '008'), (np.float64(-10.941606460775276), '008s'), (np.float64(-10.941606460775276), '009')]


 1948. [57]  [(np.float64(-10.941667361821827), 'תחל'), (np.float64(-10.941667361821827), 'תיירות'), (np.float64(-10.941667361821827),